# Introduction to Cross-Validation - Lab

## Introduction

In this lab, you'll be able to practice your cross-validation skills!


## Objectives

You will be able to:

- Compare the results with normal holdout validation
- Apply 5-fold cross validation for regression

## Let's get started

This time, let's only include the variables that were previously selected using recursive feature elimination. We included the code to preprocess below.

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.datasets import load_boston

boston = load_boston()

boston_features = pd.DataFrame(boston.data, columns = boston.feature_names)
b = boston_features["B"]
logdis = np.log(boston_features["DIS"])
loglstat = np.log(boston_features["LSTAT"])

# minmax scaling
boston_features["B"] = (b-min(b))/(max(b)-min(b))
boston_features["DIS"] = (logdis-min(logdis))/(max(logdis)-min(logdis))

#standardization
boston_features["LSTAT"] = (loglstat-np.mean(loglstat))/np.sqrt(np.var(loglstat))

In [13]:
X = boston_features[['CHAS', 'RM', 'DIS', 'B', 'LSTAT']]
y = pd.DataFrame(boston.target,columns = ['target'])
type(X)

pandas.core.frame.DataFrame

## Train test split

Perform a train-test-split with a test set of 0.20.

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

In [16]:
print(len(X_train), len(X_test), len(y_train), (y_test))

404 102 404      target
172    23.1
424    11.7
347    23.1
65     23.5
110    21.7
40     34.9
411    17.2
290    28.5
470    19.9
117    19.2
52     25.0
338    20.6
75     21.4
240    22.0
462    19.5
214    23.7
321    23.1
365    27.5
359    22.6
151    19.6
285    22.0
195    50.0
45     19.3
362    20.8
30     12.7
381    10.9
253    42.8
269    20.7
360    25.0
158    24.3
..      ...
248    24.5
445    11.8
132    23.0
218    21.5
426    10.2
67     22.0
380    10.4
483    21.8
396    12.5
334    20.7
209    20.0
31     14.5
0      24.0
313    21.6
276    33.2
81     23.9
309    20.3
142    13.4
237    31.5
335    21.1
66     19.4
38     24.7
302    26.4
71     21.7
229    31.5
140    14.0
95     28.4
101    26.5
138    13.3
154    17.0

[102 rows x 1 columns]


Fit the model and apply the model to the make test set predictions

In [17]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()

linreg.fit(X_train, y_train)
y_hat_test = linreg.predict(X_test)

Calculate the residuals and the mean squared error

In [18]:
from sklearn.metrics import mean_squared_error
test_residuals = y_hat_test - y_test

test_mse = mean_squared_error(y_test, y_hat_test)
test_mse

20.175603795417782

## Cross-Validation: let's build it from scratch!

### Create a cross-validation function

Write a function k-folds that splits a dataset into k evenly sized pieces.
If the full dataset is not divisible by k, make the first few folds one larger then later ones.

We want the folds to be a list of subsets of data!

In [21]:
def kfolds(data, k):
    #Force data as pandas dataframe
    data = pd.DataFrame(data)
    num_observations = len(data)
    fold_size = num_observations//k
    leftovers = num_observations%k
    folds = []
    start_obs = 0
    for fold_n in range(1,k+1):
        if fold_n <= leftovers:
            #Fold Size will be 1 larger to account for leftovers
            fold =  data.iloc[start_obs : start_obs+fold_size+1] 
            folds.append(fold)
            start_obs +=  fold_size + 1
        else:
            fold =  data.iloc[start_obs : start_obs+fold_size] 
            folds.append(fold)
            start_obs +=  fold_size
            
        
    # add 1 to fold size to account for leftovers
    
    return folds

### Apply it to the Boston Housing Data

In [23]:
# Make sure to concatenate the data again
bos_data = pd.concat([X.reset_index(drop=True), y], axis=1)

In [24]:
bos_folds = kfolds(bos_data, 5)

### Perform a linear regression for each fold, and calculate the training and test error

Perform linear regression on each and calculate the training and test error.

In [26]:
test_errs = []
train_errs = []
k=5

for n in range(k):
    # Split in train and test for the fold
    train = pd.concat([fold for i, fold in enumerate(bos_folds) if i!=n])
    test = bos_folds[n]
    # Fit a linear regression model
    linreg.fit(train[X.columns], train[y.columns])
    #Evaluate Train and Test Errors
    y_hat_train = linreg.predict(train[X.columns])
    y_hat_test = linreg.predict(test[X.columns])
    train_residuals = y_hat_train - train[y.columns]
    test_residuals = y_hat_test - test[y.columns]
    train_errs.append(np.mean(train_residuals.astype(float)**2))
    test_errs.append(np.mean(test_residuals.astype(float)**2))
print(train_errs)
print(test_errs)

[target    24.195577
dtype: float64, target    23.032087
dtype: float64, target    19.745073
dtype: float64, target    15.317101
dtype: float64, target    22.329973
dtype: float64]
[target    13.405145
dtype: float64, target    17.444017
dtype: float64, target    37.032711
dtype: float64, target    58.279544
dtype: float64, target    26.097989
dtype: float64]


## Cross-Validation using Scikit-Learn

This was a bit of work! Now, let's perform 5-fold cross-validation to get the mean squared error through scikit-learn. Let's have a look at the five individual MSEs and explain what's going on.

In [28]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

cv_5_results = cross_val_score(linreg, X, y, cv=5, scoring="neg_mean_squared_error")

Next, calculate the mean of the MSE over the 5 cross-validations and compare and contrast with the result from the train-test-split case.

In [29]:
cv_5_results

array([-13.40514492, -17.4440168 , -37.03271139, -58.27954385,
       -26.09798876])

##  Summary 

Congratulations! You now practiced your knowledge on k-fold crossvalidation!